# Data Processing

### Step 1: Load Libraries

In [1]:
'''
This Juputer notebook performs data transformation tasks, including 
- Converting all message JSON files into a single CSV file
- Cleaning and filtering data
- Extracting emojis from messages
- Adding datetime components
- Calculating character and emoji counts
- Performing sentiment analysis using VADER

The end result is a CSV file of all Instagram messages I have ever sent, enriched with sentiment and count data for analysis.

I created additional CSV files for 
- unigram counts
- bigram counts
- trigram counts
- emoji counts
- multi-letter word counts

This analysis is done entirely without AI, with the small exception of consulting ChatGPT 5 for emoji unicoding advice and regex pattern syntax.
'''
import pandas as pd
import os
import json
import re
import nltk
from collections import Counter
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\yeahs\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

### Step 2: Parse Messages Data and Convert to CSV

In [2]:
inbox_path = 'inbox'
all_data = []

# Iterate through inbox directories and process message_1.json files
for root, dirs, files in os.walk(inbox_path):
    for file in files:
        if file == 'message_1.json':
            file_path = os.path.join(root, file)

            # Extract recipient name
            recipient_name = os.path.basename(root).split('_')[0]

            # Open and read JSON file
            with open(file_path, 'r', encoding='utf-8') as f:
                content = json.load(f)

                if isinstance(content, dict):
                    if 'messages' in content:
                        content = content['messages']
                    else:
                        content = [content]

                # Normalize JSON data into a DataFrame
                df = pd.json_normalize(content)
                df['recipient_name'] = recipient_name  
                all_data.append(df)

# Combine all DataFrames and clean data
merged_df = pd.concat(all_data, ignore_index=True)
merged_df = merged_df[merged_df['sender_name'] == 'Siddharth Chattoraj']
merged_df = merged_df[['recipient_name', 'timestamp_ms', 'content']]
merged_df.sort_values(by='timestamp_ms', inplace=True)
merged_df.to_csv('data/all_messages.csv', index=False)

### Step 3: Clean and Extract Data + Conduct Sentiment Analysis

In [3]:
all_messages = pd.read_csv('data/all_messages.csv')

# Clean emoji encoding issues
def fix_emoji(text):
    try:
        return text.encode('latin1').decode('utf-8')
    except Exception:
        return text
all_messages['content'] = all_messages['content'].apply(fix_emoji)
all_messages = all_messages.dropna(subset=['content']).reset_index(drop=True)
allowed_chars = r"a-zA-Z0-9\s.,!?;’:'\"()\[\]{}\-\–—_@#$/\\%&*+=<>~`|^…\u00C0-\u017F"
remove_symbols_re = re.compile(fr"[^{allowed_chars}]")
extract_symbols_re = re.compile(fr"[^{allowed_chars}]")

all_messages['emojis'] = all_messages['content'].apply(
    lambda x: extract_symbols_re.findall(x) if isinstance(x, str) else []
)
bad_emojis = {'️', '“', '”', '⃣', '\u200d'}
all_messages['emojis'] = all_messages['emojis'].apply(
    lambda x: [e for e in x if e not in bad_emojis]
)
all_messages['content'] = all_messages['content'].apply(
    lambda x: ''.join([c for c in x if c not in bad_emojis]) if isinstance(x, str) else x
)

# Fix emoji color issue
def restore_color(e):
    if not isinstance(e, str):
        return e
    e = e.replace("🇸", "").replace("🇺", "")
    if e in {"❤", "☹", "☺", "☘", "♥"} and not e.endswith("\ufe0f"):
        e = e + "\ufe0f"
    return e

all_messages['emojis'] = all_messages['emojis'].apply(
    lambda lst: [restore_color(e) for e in lst] if isinstance(lst, list) else lst
)
# drop empty/whitespace emoji tokens in the lists
all_messages['emojis'] = all_messages['emojis'].apply(
    lambda lst: [e for e in lst if isinstance(e, str) and e.strip()] if isinstance(lst, list) else lst
)

def restore_color_text(s):
    if not isinstance(s, str):
        return s
    return (
        s.replace("❤", "❤️")
         .replace("☹", "☹️")
         .replace("☺", "☺️")
         .replace("☘", "☘️")
         .replace("🇸", "")
         .replace("🇺", "")
    )

all_messages['content'] = all_messages['content'].apply(restore_color_text)

# Timestamps to datetime and extract components
all_messages['datetime'] = pd.to_datetime(all_messages['timestamp_ms'], unit='ms')
all_messages = all_messages.drop(columns=['timestamp_ms'])
all_messages = all_messages.sort_values(by='datetime').reset_index(drop=True)
all_messages['year'] = all_messages['datetime'].dt.year
all_messages['month'] = all_messages['datetime'].dt.month
all_messages['day'] = all_messages['datetime'].dt.day
all_messages['hour'] = all_messages['datetime'].dt.hour

# Add count of number of words and number of emojis
all_messages['word_count'] = all_messages['content'].apply(
    lambda x: len(x.split()) if isinstance(x, str) else 0
)
all_messages['character_count'] = all_messages['content'].apply(
    lambda x: len(x) if isinstance(x, str) else 0
)

all_messages['emoji_count'] = all_messages['emojis'].apply(len)

# Sentiment Analysis using VADER
sid = SentimentIntensityAnalyzer()
all_messages['compound_sentiment'] = all_messages['content'].apply(
    lambda x: sid.polarity_scores(x)['compound']
)

def label_sentiment(score):
    if score >= 0.05:
        return 'positive'
    elif score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

all_messages['sentiment_label'] = all_messages['compound_sentiment'].apply(label_sentiment)
all_messages.to_csv('data/cleaned_messages.csv', index=False)

### Step 4: Unigrams, Bigrams, Trigrams, Emojis, and Multi-letter Words

In [4]:
cleaned_messages = pd.read_csv('data/cleaned_messages.csv')

# Define cleaning
url_pattern = r"(https?://\S+|www\.\S+)"
punctuation_pattern = r"(^[^\w']+|[^\w']+$)"

def clean_for_ngrams(text):
    if not isinstance(text, str):
        return ''
    text = text.lower()
    text = re.sub(url_pattern, '', text)
    tokens = [re.sub(punctuation_pattern, '', t) for t in text.split()]
    return ' '.join(t for t in tokens if t)

cleaned_messages['content_further'] = cleaned_messages['content'].astype(str).apply(clean_for_ngrams)

# Unigrams, Bigrams, and Emoji Frequency Counts
unigram_counts = cleaned_messages['content_further'].str.split(expand=True).stack().value_counts()
unigram_counts = pd.DataFrame(unigram_counts).rename(columns={0: 'count'}).reset_index().rename(columns={'index': 'unigram'})
unigram_counts.to_csv('data/unigram_counts.csv', index=False)

bigrams = cleaned_messages['content_further'].apply(lambda x: [' '.join(b) for b in zip(x.split()[:-1], x.split()[1:])])
bigram_counts = Counter([b for sublist in bigrams for b in sublist])
bigram_counts = pd.Series(bigram_counts).sort_values(ascending=False)
bigram_counts = pd.DataFrame(bigram_counts).rename(columns={0: 'count'}).reset_index().rename(columns={'index': 'bigram'})
bigram_counts.to_csv('data/bigram_counts.csv', index=False)

trigrams = cleaned_messages['content_further'].apply(lambda x: [' '.join(t) for t in zip(x.split()[:-2], x.split()[1:-1], x.split()[2:])])
trigram_counts = Counter([t for sublist in trigrams for t in sublist])
trigram_counts = pd.Series(trigram_counts).sort_values(ascending=False)
trigram_counts = pd.DataFrame(trigram_counts).rename(columns={0: 'count'}).reset_index().rename(columns={'index': 'trigram'})
trigram_counts.to_csv('data/trigram_counts.csv', index=False)

emoji_counts = pd.Series([e for lst in cleaned_messages['emojis'].dropna() for e in eval(lst) if isinstance(eval(lst), list)]).value_counts()
emoji_counts = pd.DataFrame(emoji_counts).rename(columns={0: 'count'}).reset_index().rename(columns={'index': 'emoji'})
emoji_counts.to_csv('data/emoji_counts.csv', index=False)

# Multi-letter analysis
triple_pattern = re.compile(r'(([a-z])\2{2,})', re.IGNORECASE)

def analyze_word(word):
    if not isinstance(word, str) or not word:
        return None
    repeats = [(match.group(1), match.group(2)) for match in triple_pattern.finditer(word)]
    if not repeats:
        return None
    longest_run, repeated_char = max(repeats, key=lambda t: len(t[0]))
    return (word, repeated_char.lower(), len(longest_run))

words = cleaned_messages['content_further'].fillna('').str.split().explode()
analyzed_words = words.apply(analyze_word).dropna()

word_data = pd.DataFrame(analyzed_words.tolist(), columns=['word', 'repeated_char', 'duplicate_length'])
word_summary = word_data.groupby('word').agg(
    occurrences=('word', 'size'),
    duplicate_length=('duplicate_length', 'max')
).reset_index()

max_duplicate_indices = word_data.groupby('word')['duplicate_length'].idxmax()
dominant_letters = word_data.loc[max_duplicate_indices, ['word', 'repeated_char']]

results = (
    word_summary.merge(dominant_letters, on='word', how='left')
    .sort_values(['occurrences', 'duplicate_length'], ascending=[False, False])
    .reset_index(drop=True)
)

results.to_csv('data/multi_letter_words.csv', index=False)